In [1]:
# randomNG = 'Philox'
# randomNG = 'PCG64DXSM'
# randomNG = 'MT19937'
# randomNG = 'SFC64'
import importlib

pass_abc ='a'

In [2]:
import numpy as np
import time
import datetime
import numba
import concurrent.futures
from math import lgamma, floor
import numpy.random
# bitNG = getattr(numpy.random, randomNG)
from numpy.random import Generator, SeedSequence
from ill_confirm_bias_FNs_03_t0FAST_03cnt_vdis_OPENminded import confirm_bias_full
np.set_printoptions(suppress=True)



In [3]:
# mp.set_start_method("fork")
start = time.perf_counter()

runs = 1000
numarms = 2
maxalphabeta = 4
# numagents = 50
# minagents = 4
# maxagents = 12
pulls = 1000
payprobs = np.asarray([.499, .5])
tollerance = 0
nsteps = 10
timesteps = 10**5

mem_depth = [5, 10, 100, 1000]
mem_snap = [10, 100, 1000, 10000] #these values must be multiples of nsteps to get recorded


networktype = 'errandom'
erprobA = 0.5
erprobB = 2-2*(1-erprobA)**.5
print(f'erprobB = {erprobB}')

mem = 100 # must be greater than max numagents. FULL STOP? if memshare = 0
memshare = 40
memstore = 20
# variablediscount = 100

#number of threads
thrds = 10
# numba.set_num_threads(thrds)

for randomNG in ['Philox']:
    bitNG = getattr(numpy.random, randomNG)
    for variablediscount in [1, 10]:

        for numagents in [4, 6, 9, 12, 15, 25]:
            sg = SeedSequence()
            rgs = [Generator(bitNG(s)) for s in sg.spawn(runs)]
            final_mp = np.zeros((runs, 2))
            depth_mp = np.zeros((runs, len(mem_snap)+1, len(mem_depth)))
            print('*********************************************')
            print('*********************************************')

            inputs = []

            with concurrent.futures.ProcessPoolExecutor(max_workers=thrds) as executor:
                future_to_confirm_bias_full = {executor.submit(confirm_bias_full, variablediscount, mem_snap, mem_depth, erprobB, mem, memshare, memstore, networktype, maxalphabeta, pulls, numagents, numarms, tollerance, payprobs, nsteps, timesteps, rgs, r): inputs for r in range(runs)}
                for future in concurrent.futures.as_completed(future_to_confirm_bias_full):
                    inputs = future_to_confirm_bias_full[future]
                    try:
                        data = future.result()
                    except Exception as exc:
                        print(f'generated an exception: y? and {exc}')
                    else:
                        final_mp[data[3]] = data[0:2]
                        depth_mp[data[3]] = data[2]


            final_mp = np.asarray(final_mp)
            suc_mp = np.asarray(final_mp[:,0])
            con_mp = np.asarray(final_mp[:,1])


            np.save(f'OPEN{pass_abc}{randomNG}vdisc{variablediscount}t0fastNGconsensus_{networktype}_ifer{erprobA}_size{numagents}_t{tollerance}_mem_s_d_{mem}_{memshare}_{memstore}.npy', con_mp)
            np.save(f'OPEN{pass_abc}{randomNG}vdisc{variablediscount}t0fastNGsuccess_{networktype}_ifer{erprobA}_size{numagents}_t{tollerance}_mem_s_d_{mem}_{memshare}_{memstore}.npy', suc_mp)
            np.save(f'OPEN{pass_abc}{randomNG}vdisc{variablediscount}t0fastNGdepths_{networktype}_ifer{erprobA}_size{numagents}_t{tollerance}_mem_s_d_{mem}_{memshare}_{memstore}.npy', depth_mp)

            sucrate = np.sum(suc_mp)/runs
            conrate = np.sum(con_mp)/runs

            depth_mean = np.mean(depth_mp, axis=0)
            depth_std = np.std(depth_mp, axis=0)

            print('time:')
            print(datetime.datetime.now())

            print(f'M memory = {mem} S memshare = {memshare} D memstore = {memstore} discount factor = {variablediscount}')
            print(f'for # agents = {numagents} success rate = {sucrate} consensus rate = {conrate}')
            print(f'using {randomNG} generator')
            print('network average memory depths:')
            print(depth_mean)
            print('depths standard deviation')
            print(depth_std)
            print(' ')
            print(' ')

finish = time.perf_counter()
print(f'Finished in {round(finish-start,0)/60} minutes')


erprobB = 0.5857864376269049
*********************************************
*********************************************
time:
2022-12-11 17:01:32.018740
M memory = 100 S memshare = 40 D memstore = 20 discount factor = 1
for # agents = 4 success rate = 0.56 consensus rate = 1.0
using Philox generator
network average memory depths:
[[323.2499319 320.8083923 282.0088576 148.7103093]
 [  5.853       0.          0.          0.       ]
 [224.754     221.972       0.          0.       ]
 [320.538     318.088     279.04        0.       ]
 [323.097     320.481     282.213     149.143    ]]
depths standard deviation
[[92.69446681 92.06555344 83.32886894 80.35345029]
 [ 5.66174805  0.          0.          0.        ]
 [93.94870666 93.22766336  0.          0.        ]
 [94.00994924 93.34902386 85.19519001  0.        ]
 [93.16920946 92.49991156 84.84327688 83.06373788]]
 
 
*********************************************
*********************************************
time:
2022-12-11 17:12:26.042546


KeyboardInterrupt: 